In [0]:
application_id = "bada1f1e-8aaa-4bb1-8d1b-72b6101283f6"
directory_id = "4249dcf4-f4a1-44f9-940d-14b50a777dd8"
#display(dbutils.secrets.listScopes())
#display(dbutils.secrets.list(scope="practice"))
service_credential = dbutils.secrets.get(scope="practice", key="adls-key")
print(service_credential)

spark.conf.set("fs.azure.account.auth.type.adls12s.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.adls12s.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.adls12s.dfs.core.windows.net", application_id)
spark.conf.set("fs.azure.account.oauth2.client.secret.adls12s.dfs.core.windows.net", service_credential)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.adls12s.dfs.core.windows.net", f"https://login.microsoftonline.com/{directory_id}/oauth2/token")

configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": application_id,
          "fs.azure.account.oauth2.client.secret": service_credential,
          "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{directory_id}/oauth2/token"}


def checkIfMounted(mounts):
    if(any(mount.mountPoint == "/mnt/adls12s/gold/" for mount in mounts )):
        print("Mount point exists")
        dbutils.fs.unmount("/mnt/adls12s/gold/")
        print("Unmounted Successfully")
        dbutils.fs.mount(source = "abfss://gold@adls12s.dfs.core.windows.net/", mount_point = "/mnt/adls12s/gold/", extra_configs = configs)
        print("Mounted Successfully")
        print(f"Mount succeeded: {mountsList}")
    else:
        print("Mount point does not exist")
        dbutils.fs.mount(source = "abfss://gold@adls12s.dfs.core.windows.net/", mount_point = "/mnt/adls12s/gold/", extra_configs = configs)
        print("Mounted Successfully")
        print(f"Mount succeeded: {mountsList}")

try:
    mountsList = dbutils.fs.mounts()
    dbutils.fs.mount(source="abfss://gold@adls12s.dfs.core.windows.net/", mount_point="/mnt/adls12s/gold/", extra_configs=configs)
    print(f"Mount succeeded: {mountsList}")
except Exception as e:
    print(f"Mount Point Exists, Unmounting and Re-Mounting\n Mount failed: {e}\n")
    checkIfMounted(mountsList)



[REDACTED]
Mount Point Exists, Unmounting and Re-Mounting
 Mount failed: An error occurred while calling o435.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/adls12s/gold; nested exception is: 
	java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/adls12s/gold
	at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:148)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:70)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.createOrUpdateMount(DBUtilsCore.scala:1142)
	at com.databricks.backend.daemon.dbutils.DBUtilsCore.$anonfun$mount$1(DBUtilsCore.scala:1168)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperation$1(UsageLogging.scala:527)
	at com.databricks.logging.UsageLogging.executeThunkAndCaptureResultTags$1(UsageLogging.scala:631)
	at com.databricks.logging.UsageLogging.$anonfun$recordOperationWith

In [0]:
#Reading data from ADLS
df = spark.read.format("delta").load("abfss://silver@adls12s.dfs.core.windows.net/airbnb_cleaned_data/")
display(df.head(5))
df.toPandas().shape
df.createOrReplaceTempView("airbnb_data")

id,host_id,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,availability_365
35732847,102263916,Staten Island,Tompkinsville,40.632808685302734,-74.09131622314453,Private room,50,1,4,2019-07-07,4.0,84
35619073,256966373,Staten Island,St. George,40.64331817626953,-74.07811737060547,Entire home/apt,100,3,1,2019-06-20,1.0,201
35489384,236186921,Staten Island,Tottenville,40.50640869140625,-74.2305908203125,Entire home/apt,75,1,1,2019-06-28,1.0,299
35455986,63227531,Staten Island,Randall Manor,40.628448486328125,-74.12294006347656,Entire home/apt,99,2,1,2019-06-20,1.0,342
35224304,43099837,Staten Island,Grymes Hill,40.61568832397461,-74.08956909179688,Entire home/apt,125,2,1,2019-06-28,1.0,44


In [0]:
#Check the average price and number of hosts in each neighbourhood
neighbourhood_df = spark.sql("SELECT COUNT(host_id) AS num_hosts, neighbourhood_group, avg(price) AS avg_price FROM airbnb_data GROUP BY neighbourhood_group ORDER BY avg(price) DESC")
neighbourhood_df.show()

+---------+-------------------+------------------+
|num_hosts|neighbourhood_group|         avg_price|
+---------+-------------------+------------------+
|    16564|          Manhattan|  180.192948563149|
|    16395|           Brooklyn|121.48752668496493|
|     4538|             Queens| 95.83494931687969|
|      311|      Staten Island| 90.42443729903538|
|      864|              Bronx| 79.74768518518519|
+---------+-------------------+------------------+



In [0]:
#Create a Pivot table for room_type, neighbourhood_group and price
pivot_df = spark.sql("SELECT neighbourhood_group, room_type, AVG(price) AS avg_price FROM airbnb_data GROUP BY neighbourhood_group, room_type")
pivot_df = pivot_df.groupBy("neighbourhood_group").pivot("room_type").agg({"avg_price": "avg"})
pivot_df.show()

+-------------------+------------------+------------------+-----------------+
|neighbourhood_group|   Entire home/apt|      Private room|      Shared room|
+-------------------+------------------+------------------+-----------------+
|             Queens|139.30791450028886| 69.27146084337349|64.71523178807946|
|           Brooklyn|171.30928975178176| 73.36387598845236|45.45862068965517|
|      Staten Island|125.36666666666666| 57.87820512820513|             57.6|
|          Manhattan|230.26423827731938|106.63329081632654| 77.8135593220339|
|              Bronx|121.33114754098361| 57.06976744186046|56.93023255813954|
+-------------------+------------------+------------------+-----------------+



In [0]:
#finding out listings for each neighbourhood
spark.sql("SELECT neighbourhood, COUNT(neighbourhood) as num_listings from airbnb_data GROUP BY neighbourhood ORDER BY COUNT(neighbourhood) DESC").show()

+------------------+------------+
|     neighbourhood|num_listings|
+------------------+------------+
|      Williamsburg|        3157|
|Bedford-Stuyvesant|        3132|
|            Harlem|        2190|
|          Bushwick|        1937|
|    Hell's Kitchen|        1520|
|      East Village|        1488|
|   Upper West Side|        1481|
|   Upper East Side|        1399|
|     Crown Heights|        1261|
|           Midtown|         984|
|       East Harlem|         938|
|        Greenpoint|         866|
|           Chelsea|         825|
|   Lower East Side|         738|
|Washington Heights|         717|
|           Astoria|         707|
|      West Village|         609|
|Financial District|         509|
|          Flatbush|         494|
|      Clinton Hill|         479|
+------------------+------------+
only showing top 20 rows



In [0]:
%python
# Number of Unique values for each category - Neighbourhood, Neighbourhood_Group, Room_type
spark.sql("SELECT (SELECT COUNT(DISTINCT neighbourhood) FROM airbnb_data) AS unique_neighbourhoods, (SELECT COUNT(DISTINCT neighbourhood_Group) FROM airbnb_data) AS unique_neighbourhood_groups, (SELECT COUNT(DISTINCT room_type) FROM airbnb_data) AS unique_room_types").show()

from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml import Pipeline

# Indexing the string columns
indexer = StringIndexer(inputCols=['neighbourhood_group', 'neighbourhood', 'room_type'],
                        outputCols=['neighbourhood_group_indexed', 'neighbourhood_indexed', 'room_type_indexed'])

# OneHotEncoding the indexed columns
encoder = OneHotEncoder(inputCols=['neighbourhood_group_indexed', 'neighbourhood_indexed', 'room_type_indexed'], 
                        outputCols=['neighbourhood_group_encoded', 'neighbourhood_encoded', 'room_type_encoded'])

# Creating a pipeline to apply both transformations
pipeline = Pipeline(stages=[indexer, encoder])

# Fitting the pipeline to the dataframe
model = pipeline.fit(df)
df_encoded = model.transform(df)

# Display the transformed dataframe
display(df_encoded.head(5))

df_encoded.printSchema()

+---------------------+---------------------------+-----------------+
|unique_neighbourhoods|unique_neighbourhood_groups|unique_room_types|
+---------------------+---------------------------+-----------------+
|                  218|                          5|                3|
+---------------------+---------------------------+-----------------+



id,host_id,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,availability_365,neighbourhood_group_indexed,neighbourhood_indexed,room_type_indexed,neighbourhood_group_encoded,neighbourhood_encoded,room_type_encoded
19343700,135522817,Manhattan,Upper East Side,40.77412033081055,-73.95694732666016,Entire home/apt,198,1,143,2019-06-15,5.909999847412109,300,0.0,7.0,0.0,"Map(vectorType -> sparse, length -> 4, indices -> List(0), values -> List(1.0))","Map(vectorType -> sparse, length -> 217, indices -> List(7), values -> List(1.0))","Map(vectorType -> sparse, length -> 2, indices -> List(0), values -> List(1.0))"
19334916,7727013,Manhattan,Harlem,40.809791564941406,-73.94288635253906,Private room,200,3,2,2018-09-17,0.1599999964237213,24,0.0,2.0,1.0,"Map(vectorType -> sparse, length -> 4, indices -> List(0), values -> List(1.0))","Map(vectorType -> sparse, length -> 217, indices -> List(2), values -> List(1.0))","Map(vectorType -> sparse, length -> 2, indices -> List(1), values -> List(1.0))"
19333921,796111,Manhattan,West Village,40.73807907104492,-74.0025405883789,Entire home/apt,150,30,3,2018-05-29,0.1599999964237213,83,0.0,16.0,0.0,"Map(vectorType -> sparse, length -> 4, indices -> List(0), values -> List(1.0))","Map(vectorType -> sparse, length -> 217, indices -> List(16), values -> List(1.0))","Map(vectorType -> sparse, length -> 2, indices -> List(0), values -> List(1.0))"
19330704,134370506,Manhattan,Morningside Heights,40.81237030029297,-73.95984649658203,Private room,80,1,39,2019-07-06,1.559999942779541,13,0.0,36.0,1.0,"Map(vectorType -> sparse, length -> 4, indices -> List(0), values -> List(1.0))","Map(vectorType -> sparse, length -> 217, indices -> List(36), values -> List(1.0))","Map(vectorType -> sparse, length -> 2, indices -> List(1), values -> List(1.0))"
19329734,135401519,Manhattan,Kips Bay,40.740631103515625,-73.98204803466797,Entire home/apt,325,30,155,2019-06-19,6.21999979019165,240,0.0,26.0,0.0,"Map(vectorType -> sparse, length -> 4, indices -> List(0), values -> List(1.0))","Map(vectorType -> sparse, length -> 217, indices -> List(26), values -> List(1.0))","Map(vectorType -> sparse, length -> 2, indices -> List(0), values -> List(1.0))"


root
 |-- id: integer (nullable = true)
 |-- host_id: integer (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: float (nullable = true)
 |-- longitude: float (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- minimum_nights: integer (nullable = true)
 |-- number_of_reviews: integer (nullable = true)
 |-- last_review: date (nullable = true)
 |-- reviews_per_month: float (nullable = true)
 |-- availability_365: integer (nullable = true)
 |-- neighbourhood_group_indexed: double (nullable = false)
 |-- neighbourhood_indexed: double (nullable = false)
 |-- room_type_indexed: double (nullable = false)
 |-- neighbourhood_group_encoded: vector (nullable = true)
 |-- neighbourhood_encoded: vector (nullable = true)
 |-- room_type_encoded: vector (nullable = true)



In [0]:
df.write.format("delta").mode('overwrite').partitionBy("neighbourhood_group").save("/mnt/adls12s/gold/airbnb_staged_data")